## DiamondEdgeFit.ipynb

Copyright &copy; 2025 Pease et al.

### About
This is a python software that will fit diamond edge data using flat, bevel, and tDAC anvils.   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Smoothing is required to effecitivly fit the torodial DAC.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Smoothing is not generally need to fit bevel or flat anvils

For questions regarding this software, contact:  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Allison M. Pease: peaseall@msu.edu

In [ ]:
# Importing the necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit
from scipy.misc import derivative
import csv
from datetime import datetime
from scipy.signal import savgol_filter

## Data Location

In [ ]:
# Folder and File Names and Locations
    # Path to the original directory inputed by the user (Old_Folder) and the new directory (New_Folder)
Path = "C:/Users"
    # Remember to add .txt to the end of the file name
File_Name = "File_Name.txt"

In [ ]:
    # Remember to add .csv to the end of the file name
CSV_File = "CSV_File.csv"
# Check if the CSV file exists and write the header if it doesn't
try:
    with open(csv_file, 'x', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Data Name', 'Reference', 'Position', 'Akahama Pressure (GPa)', 'Eremets Pressure (GPa)','Range Reference Min','Range Reference Max','Range Edge Min','Range Edge Max','Window', 'poly order','Timestamp'])
except FileExistsError:
    pass

## Load Data

In [ ]:
# Load Data
def read_data(File_Name):
    data = np.loadtxt(File_Name)
    x = data[:, 0]
    y = data[:, 1]
    return x, y
x, y = read_data(data)

## Smooth Data

In [ ]:
# Define the ranges for finding max and min derivatives
range_max = (1320, 1338)
#change Each time
range_min = (1620,1645) #(1360, 1370) 
#Normalize data to avoid changing graph scaling
def normalize_data(data):
    max_value = max(data)
    return [value / max_value for value in data]
y = normalize_data(y)
#smooth data in y
window_length = 31  # possible range: 21-51, must be an odd number
polyorder = 11      # Polynomial order 13-7
y_smooth = savgol_filter(y, window_length, polyorder)
# Plot the original data and the smoothed data
plt.figure(figsize=(10, 6))
plt.xlim(1300,1700)
plt.ylim(0,1.3)
plt.plot(x, y, label='Data', color='blue')
plt.plot(x, y_smooth, label='Smoothed Data', color='black')
plt.xlabel('cm-1')
plt.legend()
plt.title('Data and smoothed fit')
plt.show()

## Derivative and Gaussian Fit

In [ ]:
#Run this to fit the smoothed data 
#Take the first derivative
dy_dx = np.gradient(y, x)
dy_dx_smooth = np.gradient(y_smooth, x)
# Find indices within the specified ranges
indices_max = np.where((x >= range_max[0]) & (x <= range_max[1]))[0]
indices_min = np.where((x >= range_min[0]) & (x <= range_min[1]))[0]
# Gaussian function
def gaussian(x, amp, cen, wid):
    return amp * np.exp(-(x - cen)**2 / (2 * wid**2))
# Fit Gaussian to the first derivative in the given ranges
#the reference is never smoothed, the gaussian is fit to the data 
popt_max, _ = curve_fit(gaussian, x[indices_max], dy_dx[indices_max], p0=[1, np.mean(x[indices_max]), 1])
# evaluate if the edge position needs to be smoothed to fit the gaussian to the data 
#smooth the DE  position (default) 
popt_min, _ = curve_fit(gaussian, x[indices_min], dy_dx_smooth[indices_min], p0=[1, np.mean(x[indices_min]), 1])
#Do not smooth the DE  position 
#popt_min, _ = curve_fit(gaussian, x[indices_min], dy_dx[indices_min], p0=[1, np.mean(x[indices_min]), 1])
# Determine the maximum and minimum derivative values based on the Gaussian fit
max_dy_dx = gaussian(x[indices_max], *popt_max).max()
min_dy_dx = gaussian(x[indices_min], *popt_min).min()
# Find the x locations of the maximum and minimum derivative values
x_max_dy_dx = x[indices_max][np.argmax(gaussian(x[indices_max], *popt_max))]
x_min_dy_dx = x[indices_min][np.argmin(gaussian(x[indices_min], *popt_min))]
# Plot the derivative and the gaussian fit
plt.figure(figsize=(10, 6))
plt.xlim(1300,1700)
plt.ylim(-.5,.5)
plt.scatter(x, y_smooth, label='Smoothed Data', color='black')
plt.plot(x, dy_dx * 5, label='Derivative', color='green')
plt.plot(x, dy_dx_smooth * 5, label='Derivative from smoothed data', color='blue')
plt.plot(x[indices_max], gaussian(x[indices_max], *popt_max) * 5, label='Gaussian Fit (Reference Position)', linestyle='--', color='orange')
plt.plot(x[indices_min], gaussian(x[indices_min], *popt_min) * 5, label='Gaussian Fit (Edge Position)', linestyle='--', color='red')
plt.xlabel('cm-1')
plt.legend()
plt.title('Data and its Derivative with Gaussian Fit')
plt.savefig("%s/%s_Image" % (path, name))
print('Reference position =', x_max_dy_dx)
print('Diamond edge position =', x_min_dy_dx)

## Calculate Pressure

In [ ]:
#Akahama + 2010 (https://iopscience.iop.org/article/10.1088/1742-6596/215/1/012195/pdf).
#Constants
A=547 #GPa
A_unc=11
B=3.75
B_unc=0.2
w_ref=x_max_dy_dx
w=x_min_dy_dx
# Solve for pressure using Akahama + 2010 
Akahama10_pressure=A*(w-w_ref)/w_ref*(1+0.5*(B-1)*(w-w_ref)/w_ref)
print ('Pressure based on Akahama =', Akahama10_pressure, "GPa")
#Eremets + 2023 (https://www.nature.com/articles/s41467-023-36429-9.pdf).
#constants
A_em=517 #GPa
A_em_unc=5
B_em=764 #GPa
B_em_unc=14
# Solve for pressure using Eremets + 2023
Eremets23_pressure= A_em*(w-w_ref)/w_ref+B_em*((w-w_ref)/w_ref*(w-w_ref)/w_ref)
print ('Pressure based on Eremets =', Eremets23_pressure, "GPa")
# Get the current date and time
timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
print(timestamp)

## Save to CSV file

In [ ]:
# Check if the CSV file exists and write the header if it doesn't
try:
    with open(csv_file, 'x', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(['Data Name', 'Reference', 'Position', 'Akahama Pressure (GPa)', 'Eremets Pressure (GPa)','Range Reference Min','Range Reference Max','Range Edge Min','Range Edge Max','Window', 'poly order','Timestamp'])
except FileExistsError:
    pass
# Append the results to the CSV file
with open(csv_file, mode='a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow([name, x_max_dy_dx, x_min_dy_dx, Akahama10_pressure, Eremets23_pressure, range_max[0], range_max[1], range_min[0], range_min[1], window_length, polyorder, timestamp])
print('data saved for', name, 'at', timestamp)

Copyright &copy; 2025 Pease et al.

Last Updated: February 13<sup>th</sup>, 2025

### License
This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.